<a href="https://colab.research.google.com/github/AmedeeRoy/BirdDL/blob/main/projection/SL_unet_projection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# mount google drive
from google.colab import drive
drive.mount("/content/drive/")

%cd drive/My\ Drive/


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
[Errno 2] No such file or directory: 'drive/My Drive/'
/content/drive/My Drive


# LOAD MODELS

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

import sqlite3

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

from utils.trip import *
from utils.unet import *

# Load data

In [9]:
data_all = pd.read_table('./data/gps+5-30s.csv', sep = ',')
check = pd.read_table('./data/check_gps+5-30s.csv', sep = ',')

# SL 15s
## data

In [10]:
select = check.trip[(check.species == 'SL') & (check.res == 15)]
data = data_all[data_all.trip.isin(select)].copy()
data['dive'] = 0
data = standardize_data(data)
data

,trip,datetime,lon,lat,gaps,step_speed,step_direction,dive,lon_std,lat_std,step_speed_std,step_direction_cos,step_direction_sin
440468,F1219_1_SL_T1,2019-12-03 16:40:51,-32.390713,-3.818904,0.0,11.315025,-3.353815,0,-2.970638,0.629552,0.560116,0.998287,-0.058502
440469,F1219_1_SL_T1,2019-12-03 16:41:06,-32.390057,-3.817131,0.0,14.025071,-6.284878,0,-2.914002,0.706926,0.694269,0.993990,-0.109472
440470,F1219_1_SL_T1,2019-12-03 16:41:21,-32.389183,-3.815881,0.0,11.310344,14.638745,0,-2.838546,0.761476,0.559884,0.967538,0.252724
440471,F1219_1_SL_T1,2019-12-03 16:41:36,-32.387932,-3.815489,0.0,9.708903,37.663821,0,-2.730542,0.778583,0.480610,0.791610,0.611027
440472,F1219_1_SL_T1,2019-12-03 16:41:51,-32.386833,-3.815223,0.0,8.373400,3.799456,0,-2.635661,0.790192,0.414500,0.997802,0.066264
...,...,...,...,...,...,...,...,...,...,...,...,...,...
452455,F1219_5_SL_T2,2019-12-04 16:21:21,-32.395439,-3.815926,0.0,11.583244,-3.439604,0,-1.501518,-2.128133,0.677569,0.998199,-0.059996
452456,F1219_5_SL_T2,2019-12-04 16:21:36,-32.394314,-3.816797,0.0,10.543468,-15.300548,0,-1.492472,-2.131845,0.616747,0.964555,-0.263882
452457,F1219_5_SL_T2,2019-12-04 16:21:51,-32.393265,-3.817728,0.0,10.395203,3.843200,0,-1.484038,-2.135811,0.608074,0.997751,0.067026
452458,F1219_5_SL_T2,2019-12-04 16:22:06,-32.392310,-3.817412,0.0,7.451897,-59.990498,0,-1.476356,-2.134465,0.435904,0.500144,-0.865942


In [11]:
model = torch.load('SV_unet_matrixlonlatcov_15s.pt')
window = 20
variable = ('lon_std', 'lat_std', 'gaps')

In [12]:
dive_estim = []

for i in select:
    t = data[data.trip == i].copy()
        
    test_set = TrajDataSet(t, window, variable, transform = ToTensor())
    
    # Test the model
    estim = np.zeros(len(t))
    nb = np.zeros(len(t))
    
    list_out = []
    model.eval()
    k = 0
    with torch.no_grad():
        for (x, y, z) in test_set:

            # Run the forward pass
            out = model(x, y.unsqueeze(1))
            
            estim[k:k + round(window)] += out.cpu().squeeze().numpy()
            nb[k:k + round(window)] += 1
            k+=1
    # # add to list by trajectory
    dive_estim.append(estim/nb)
    print(i)

data['prediction'] = 1/(1+np.exp(-np.hstack(dive_estim)))

F1219_1_SL_T1
F1219_1_SL_T2
F1219_1_SL_T3
F1219_2_SL_T1
F1219_3_SL_T1
F1219_3_SL_T2
F1219_3_SL_T3
F1219_3_SL_T4
F1219_4_SL_T1
F1219_4_SL_T2
F1219_5_SL_T1
F1219_5_SL_T2


In [13]:
data.to_csv('SL_15s.csv', index = False)

# SL 30s
## data

In [14]:
select = check.trip[(check.species == 'SL') & (check.res == 30)]
data = data_all[data_all.trip.isin(select)].copy()
data['dive'] = 0
data = standardize_data(data)
data

,trip,datetime,lon,lat,gaps,step_speed,step_direction,dive,lon_std,lat_std,step_speed_std,step_direction_cos,step_direction_sin
452462,F1219_6_SL_T1,2019-12-08 15:20:44,-32.387257,-3.815195,0.0,9.563439,-11.569287,0,-1.526070,-0.599290,0.545063,0.979683,-0.200553
452463,F1219_6_SL_T1,2019-12-08 15:21:14,-32.384647,-3.814265,0.0,10.262291,-0.674742,0,-1.452948,-0.577136,0.585258,0.999931,-0.011776
452464,F1219_6_SL_T1,2019-12-08 15:21:44,-32.382219,-3.813145,0.0,9.901702,-5.173295,0,-1.384953,-0.550448,0.564518,0.995927,-0.090168
452465,F1219_6_SL_T1,2019-12-08 15:22:14,-32.381168,-3.810930,0.0,9.092769,-39.843175,0,-1.355510,-0.497674,0.517992,0.767801,-0.640688
452466,F1219_6_SL_T1,2019-12-08 15:22:44,-32.380078,-3.808105,0.0,11.233361,-4.291495,0,-1.324986,-0.430349,0.641110,0.997196,-0.074831
...,...,...,...,...,...,...,...,...,...,...,...,...,...
458638,F1219_9_SL_T4,2019-12-09 15:45:05,-32.439253,-3.849310,0.0,1.302111,-52.956686,0,-0.614974,-0.542378,0.097118,0.602419,-0.798180
458639,F1219_9_SL_T4,2019-12-09 15:45:35,-32.439042,-3.849072,0.0,1.181759,32.860377,0,-0.594626,-0.503417,0.087705,0.839995,0.542594
458640,F1219_9_SL_T4,2019-12-09 15:46:05,-32.439451,-3.849464,0.0,2.100274,-175.336965,0,-0.634000,-0.567464,0.159544,-0.996690,-0.081295
458641,F1219_9_SL_T4,2019-12-09 15:46:35,-32.439032,-3.849310,0.0,1.654082,-156.271911,0,-0.593601,-0.542373,0.124647,-0.915465,-0.402397


In [15]:
model = torch.load('SV_unet_matrixlonlatcov_30s.pt')
window = 20
variable = ('lon_std', 'lat_std', 'gaps')

In [16]:
dive_estim = []

for i in select:
    t = data[data.trip == i].copy()
        
    test_set = TrajDataSet(t, window, variable, transform = ToTensor())
    
    # Test the model
    estim = np.zeros(len(t))
    nb = np.zeros(len(t))
    
    list_out = []
    model.eval()
    k = 0
    with torch.no_grad():
        for (x, y, z) in test_set:

            # Run the forward pass
            out = model(x, y.unsqueeze(1))
            
            estim[k:k + round(window)] += out.cpu().squeeze().numpy()
            nb[k:k + round(window)] += 1
            k+=1
    # # add to list by trajectory
    dive_estim.append(estim/nb)
    print(i)

data['prediction'] = 1/(1+np.exp(-np.hstack(dive_estim)))

F1219_6_SL_T1
F1219_6_SL_T2
F1219_7_SL_T1
F1219_7_SL_T2
F1219_7_SL_T3
F1219_7_SL_T4
F1219_8_SL_T1
F1219_9_SL_T1
F1219_9_SL_T2
F1219_9_SL_T3
F1219_9_SL_T4


In [17]:
data.to_csv('SL_30s.csv', index = False)